In [9]:
import pandas as pd
import datetime
from github import Github
import os
import requests
from requests.adapters import HTTPAdapter

from urllib3 import Retry
from csv import DictWriter

# Step 1: Obtain a list of relevant repositories

Used Github Search: https://seart-ghs.si.usi.ch to get repositories in Python with not less then 5 commits for the last 3 years.

In [10]:
df = pd.read_csv('git_repos_20-23.csv')
df = df.sort_values(['stargazers'], ascending=False)
df.head()

,name,isFork,commits,branches,defaultBranch,releases,contributors,license,watchers,stargazers,...,homepage,mainLanguage,totalIssues,openIssues,totalPullRequests,openPullRequests,lastCommit,lastCommitSHA,hasWiki,isArchived
15442,ultralytics/yolov5,False,2103,36,master,8,240.0,GNU General Public License v3.0,311,28567,...,https://ultralytics.com,Python,6002.0,321.0,1773.0,16.0,2022-07-08T12:32:40,526e650553819dbff67897b9c752c4072e989823,True,False
14249,z4nzu/hackingtool,False,218,1,master,0,27.0,MIT License,1003,26980,...,https://forms.gle/ntuAX8BGRR5yAb9ZA,Python,249.0,80.0,69.0,9.0,2022-12-03T04:47:23,8af26be04fd173113448347c778c7b0edde9a794,True,False
13606,mingrammer/diagrams,False,451,4,master,36,105.0,MIT License,351,26199,...,https://diagrams.mingrammer.com,Python,392.0,236.0,372.0,85.0,2022-09-13T09:11:45,834899659ae2e4f9f0d0dd9d01a4d7f31513d726,True,False
26476,babysor/mockingbird,False,164,10,main,1,33.0,Other,240,25499,...,NaN,Python,713.0,346.0,79.0,8.0,2022-12-16T03:16:25,cd20d21f3d845b2e8b8d0e65b9de5d974ccc85c4,True,False
22515,tencentarc/gfpgan,False,106,1,master,8,11.0,Other,369,25034,...,NaN,Python,259.0,166.0,36.0,10.0,2022-09-16T11:33:26,2eac2033893ca7f427f4035d80fe95b92649ac56,True,False


In [11]:
# split_names = [name.split('/') for name in df.name.to_list()]
# user_names = [name[0] for name in split_names]
# repo_names = [name[1] for name in split_names]

In [12]:
topics = ['maching-learning', 'deep-learning', 'natural-language-processing', 
          'nlp', 'cnn', 'rnn', 'gnn', 'transformer', 'attention', 'generative-adversarial-network',
          'pytorch', 'tensorflow', 'keras', 'pytorch-lightning', 'neural-network']

contents = ['import torch', 'import keras', 'import tensorflow']

descriptions = ['torch', 'keras', 'tensorflow']

In [6]:
# class GetRepositories():
#     def __init__(self, topics, contents, descriptions):
#         self.topics = topics
#         self.contents = contents
#         self.descriptions = descriptions
        
#     def check_repo(self, access_token, user_name, repo_name):
#         url = 'https://api.github.com/'
#         print(f'Query user = {user_name} repo = {repo_name}...', end=' ')
        
#         header = {'Authorization': f'Bearer {access_token}'}
        
#         if requests.get(url + 'users/' + user_name, headers=header).status_code > 400:
#             print('Failed!\nUser does not exist!\n')
#             # print(requests.get(url + 'users/' + user_name).status_code)
#             return False
        
#         if requests.get(url + 'repos/' + user_name + '/' + repo_name, headers=header).status_code > 400:
#             print('Failed!\nRepo does not exist!\n')
#             return False

#         # using an access token
#         g = Github(access_token)
#         user = g.get_user(user_name)
#         repo = user.get_repo(repo_name)

                
#         if self.check_topics(repo):
#             print(f'Success!\nFound required topics in: {repo.get_topics()}!\n')
#             return True
        
#         if self.check_description(repo):
#             print(f'Success!\nFound required description in: <{repo.description}>!\n')
#             return True
            
#         if  self.check_contents(repo):
#             print('Success!\nFound required content!\n')
#             return True
        
#         print('Failed!')
    
#         return False
                
#     def check_topics(self, repo):
#         repo_topics = repo.get_topics()

#         for topic in self.topics:
#             if topic in repo_topics:
#                 return True
#         return False
        
#     def check_description(self, repo):
#         repo_description = repo.description
        
#         if repo_description is None:
#             return False
        
#         for desc in self.descriptions:
#             if desc in repo_description:
#                 return True
#         return False
        
#     def check_contents(self, repo):
#         repo_contents = repo.get_contents('')
#         while repo_contents:
#             file_content = repo_contents.pop(0)
#             if file_content.type == "dir":
#                 repo_contents.extend(repo.get_contents(file_content.path))
#             else:
#                 file_name = file_content.path
                
#                 filename, file_extension = os.path.splitext(file_name)
#                 if file_extension in [".py"]:
#                     # print(filename)
#                     try:
#                         file_data = file_content.decoded_content.decode()
#                         for content in self.contents:
#                             if content in file_data:
#                                 return True
#                     except AssertionError:
#                         break
#                     # print(file_data)

#         return False

In [12]:
# access_token = ""

# get_ml_repos = GetRepositories(topics, contents, descriptions)
# ml_repos = []
# # get_ml_repos.check_repo(access_token, 'olyandrevn', 'web-scraper')
# # get_ml_repos.check_repo(access_token, 'olyandrevn', 'text-generator')
# # get_ml_repos.check_repo(access_token, 'strangerealintel', 'cerberus')

# for i, (user_name, repo_name) in enumerate(zip(user_names, repo_names)):
#     print(i, end=' ')
#     is_ml_repo = get_ml_repos.check_repo(access_token, user_name, repo_name)
    
#     if is_ml_repo:
#         ml_repos.append([user_name, repo_name])

In [13]:
# retries to avoid github api rate limit (403 status code)
def create_github_session() -> requests.Session:
    s = requests.Session()
    retries = Retry(total=50, backoff_factor=10, status_forcelist=[403])
    s.mount('https://', HTTPAdapter(max_retries=retries))
    return s

In [27]:
class GetRepositories():
    def __init__(self, topics, contents, descriptions):
        self.topics = topics
        self.contents = contents
        self.descriptions = descriptions
        
    def check_repo(self, access_token, session, name):
        url = 'https://api.github.com/'
        print(f'Query {name}...', end=' ')
        
        header = {'Authorization': f'Bearer {access_token}'}
        
        if session.get(url + 'repos/' + name, headers=header).status_code > 400:
            print('Failed!')
            return None

        retries = Retry(total=50, backoff_factor=10, status_forcelist=[403])
        g = Github(access_token, retry=retries)
        
        try:
            repo = g.get_repo(name)
        except Exception as e:
            print(f'Failed! {e.message}')
            return None

        repo_info = {'name': name, 'full_name': repo.full_name, 'topics': repo.get_topics(), 'description': repo.description}

        if self.check_topics(repo_info['topics']):
            print(f'Success!\nFound required topics in: {repo_info["topics"]}!\n')
            return repo_info
        
        if self.check_description(repo_info['description']):
            print(f'Success!\nFound required description in: <{repo_info["description"]}>!\n')
            return repo_info
        
        print('Failed!')
    
        return None
                
    def check_topics(self, repo_topics):
        for topic in self.topics:
            if topic in repo_topics:
                return True
        return False
        
    def check_description(self, repo_description):
        if repo_description is None:
            return False
        
        for desc in self.descriptions:
            if desc in repo_description:
                return True
        return False
        
    def check_contents(self, repo):
        repo_contents = repo.get_contents('')
        while repo_contents:
            file_content = repo_contents.pop(0)
            if file_content.type == "dir":
                repo_contents.extend(repo.get_contents(file_content.path))
            else:
                file_name = file_content.path
                
                filename, file_extension = os.path.splitext(file_name)
                if file_extension in [".py"]:
                    # print(filename)
                    try:
                        file_data = file_content.decoded_content.decode()
                        for content in self.contents:
                            if content in file_data:
                                return True
                    except AssertionError:
                        break
                        
        return False

In [18]:
access_token = ""
session = create_github_session()
names = df.name.to_list()

field_names = ['name', 'full_name', 'topics', 'description']
get_ml_repos = GetRepositories(topics, contents, descriptions)

with open('ml_repos_sorted.csv', 'a') as f_object:
    dictwriter_object = DictWriter(f_object, fieldnames=field_names)
 
    for i, name in enumerate(names):
        print(i, end=' ')
        repo_info = get_ml_repos.check_repo(access_token, session, name)
        if repo_info is not None:
            dictwriter_object.writerow(repo_info)

    f_object.close()

0 Query ultralytics/yolov5... Success!
Found required topics in: ['yolov3', 'yolov4', 'yolov5', 'object-detection', 'pytorch', 'onnx', 'coreml', 'ios', 'tflite', 'yolo', 'deep-learning', 'machine-learning', 'ml']!

1 Query z4nzu/hackingtool... Failed!
2 Query mingrammer/diagrams... Failed!
3 Query babysor/mockingbird... Success!
Found required topics in: ['ai', 'speech', 'pytorch', 'deep-learning']!

4 Query tencentarc/gfpgan... Success!
Found required topics in: ['pytorch', 'gan', 'deep-learning', 'super-resolution', 'face-restoration', 'image-restoration', 'gfpgan']!

5 Query paddlepaddle/paddleocr... Failed!
6 Query beurtschipper/depix... Failed!
7 Query github/copilot-docs... Failed!
8 Query xinntao/real-esrgan... Success!
Found required topics in: ['esrgan', 'pytorch', 'real-esrgan', 'super-resolution', 'image-restoration', 'denoise', 'jpeg-compression', 'amine']!

9 Query jaidedai/easyocr... Success!
Found required topics in: ['ocr', 'deep-learning', 'crnn', 'pytorch', 'lstm', 'm

RateLimitExceededException: 403 {"message": "API rate limit exceeded for user ID 33371372.", "documentation_url": "https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting"}

In [28]:
access_token = ""
session = create_github_session()
names = df.name.to_list()

field_names = ['name', 'full_name', 'topics', 'description']
get_ml_repos = GetRepositories(topics, contents, descriptions)

with open('ml_repos_sorted.csv', 'a') as f_object:
    dictwriter_object = DictWriter(f_object, fieldnames=field_names)
 
    for i, name in enumerate(names):
        if i >= 1000:
            print(i, end=' ')
            repo_info = get_ml_repos.check_repo(access_token, session, name)
            if repo_info is not None:
                dictwriter_object.writerow(repo_info)

    f_object.close()

1000 Query shi-labs/versatile-diffusion... Failed!
1001 Query thinklab-sjtu/awesome-ml4co... Failed!
1002 Query zenalc/algobot... Failed!
1003 Query mit-han-lab/data-efficient-gans... Success!
Found required topics in: ['gans', 'pytorch', 'tensorflow', 'data-efficient', 'generative-adversarial-network', 'image-generation', 'neurips-2020']!

1004 Query nvlabs/nvae... Failed!
1005 Query san089/udacity-data-engineering-projects... Failed!
1006 Query pettingzoo-team/pettingzoo... Failed!
1007 Query jina-ai/docarray... Success!
Found required topics in: ['docarray', 'data-structures', 'multimodal', 'cross-modal', 'neural-search', 'unstructured-data', 'deep-learning', 'nested-data', 'qdrant', 'weaviate', 'sqlite', 'nearest-neighbor-search', 'protobuf', 'graphql', 'elasticsearch', 'dataclass', 'multi-modal', 'semantic-search', 'vector-search']!

1008 Query hkchengrex/xmem... Success!
Found required topics in: ['computer-vision', 'deep-learning', 'eccv-2022', 'eccv2022', 'pytorch', 'segmentati

KeyboardInterrupt: 